# Convection-Diffusion Equation

$$
\DeclareMathOperator{\div}{div}
\DeclareMathOperator{\grad}{grad}
$$

Consider the stationary convection-diffusion problem, often jokingly referred to as the **Con-fusion Problem**. Such problems  arise in modeling phenomena with both diffusion and convection, such as heat flow.
Fourier's law of heat conduction states that heat flux $q_{\text{conductive}}$ is proportional to the negative temperature gradient: $q_{\text{conductive}} = - \kappa \grad u$, where $u$ denotes the temperature and the constant of proportionality, $\kappa$, is called the thermal conductivity. Convective heat flux takes the form $q_{\text{convective}} = \beta u$ where $\beta$ is an advection velocity vector field that transports heat along it. By conservation of energy
the heat lost through the sum of these fluxes on any control volume $V$ must equal the sum of heat sources $f$ within it:
$$
\int_{\partial V} (q_{\text{conductive}} + q_{\text{convective}}) \cdot n = \int_V f,
$$
where $n$ denotes  the outward unit normal.
Transforming this integral law into a differential one as usual, we obtain the convection-diffusion equation,  
$$
\begin{aligned}
-\div(\kappa \grad u - \beta u)  = 0,
\end{aligned}
$$
which one might also rewrite as 
$$
-\div(\kappa \grad u) +  \beta \cdot \grad u = 0,
$$
in the often-occurring case of $\div \beta=0$. There is nothing confusing about this equation when $\kappa$ and $\beta$ are nice. Confusion might come when $\beta$ is too large, or when $\kappa$ is too small, cases where the standard Lagrange finite element method can give strange solutions, as we see in this notebook. 

## A model problem

A simple model boundary value problem with the convection-diffusion equation is as follows.
$$
\DeclareMathOperator{\div}{div}
\DeclareMathOperator{\grad}{grad}
\begin{aligned}
-\div(\kappa \grad u - \beta u) & = f,  && \text { in } \Omega =\text{unit square,}
\\
u  &= g,  && \text { on left boundary}, 
\\
u & = 0  && \text { on right boundary}, 
\\
\kappa \frac{\partial u}{ \partial n} - (\beta \cdot n) u & = 0,
 && \text { on top and bottom boundaries},
\end{aligned}
$$
where $n$ is the outward unit normal.   To obtain a nice test problem we choose very smooth data (so that no tricks are needed to integrate the data accurately), namely
$$
f \equiv 0, \quad g \equiv 1/2, 
\quad \beta \equiv (1, 0).
$$
and assume that $\kappa>0$ is a constant function.



<div class="alert alert-info">
    <b><font color=red>Exercise I:</font></b>      
  Show that the exact solution $u$ (is independent of the second coordinate $y$) and is given only in terms of the first coordinate $x$ by 
$$
u(x, y) =  \frac{1 - e^{(x-1)/\kappa}} {2(1 - e^{-1/\kappa})}.
$$  
</div>

## Finite element solution

The weak formulation is derived as usual by multiplying by a test function $v$ and integrating 
by parts. Letting $\Gamma$ be the subset of $\partial\Omega$ consisting of the left and right boundary segments, let $u_g \in H^1(\Omega)$ be an extension of the boundary data $g$. Then writing
$$
u = \mathring u + u_g,
$$
we find $\mathring u \in \mathring{H}_\Gamma^1(\Omega)$ such that 
$$
\int_\Omega\kappa\grad u \cdot \grad v - \int_\Omega u \beta \cdot  \grad v = \int_\Omega f v,
$$
for all 
$v \in \mathring{H}^1_\Gamma(\Omega).$



We mesh $\Omega$ by $\Omega_h$ and consider the finite element approximation. For the above-mentioned choice of $g$,  we can obviously construct an extension $u_g$ that is in $P_p(\Omega_h) \cap H^1(\Omega)$ and the exact solution  $u$  is in $u_g \in \mathring{H}_\Gamma^1(\Omega)$. The  degree $p$ Lagrange finite element approximation $u_h$ is in $u_g + \mathring{V}^{\Gamma}_{hp}$ where 
$\mathring{V}^{\Gamma}_{hp} = P_p(\Omega_h) \cap \mathring{H}_\Gamma^1(\Omega)$.

<div class="alert alert-info">
    <b><font color=red>Exercise II:</font></b>      
   Prove that there is a mesh-independent $C>0$  such that
  $$
  \| u - u_h \|_{H^1(\Omega)} \le \;\frac{C}{\kappa} 
  \;\inf_{v_h \in u_g + \mathring{V}^\Gamma_{hp}} \| u - v_h \|_{H^1(\Omega)}.
  $$
</div>

In [ ]:
import ngsolve as ng
from ngsolve.webgui import Draw
from netgen.geom2d import unit_square
from ngsolve import grad, dx, CoefficientFunction, GridFunction, BilinearForm, LinearForm

In [ ]:
# Set convection veclocity

beta = CoefficientFunction((1, 0))

# Set Gamma = left|right and space with essential b.c.

mesh = ng.Mesh(unit_square.GenerateMesh(maxh=0.1))
V = ng.H1(mesh, order=2, dirichlet='left|right')

# Inhomogeneous Dirichlet b.c. 

g = 0.5

In [ ]:
def SolveConfusion(mesh, kappa=1, p=2):
    """Use Lagrange finite elements to approximate the above
    variational formulation of the confusion model problem."""

    V = ng.H1(mesh, order=p, dirichlet='left|right')
    u, v = V.TnT()
    
    a = BilinearForm(V)
    a+= (kappa*grad(u)*grad(v) - u*beta*grad(v)) * dx 
    f = LinearForm(V)
    uh = GridFunction(V)
    
    with ng.TaskManager():
        uh.Set(g, definedon=mesh.Boundaries('left'))
        a.Assemble()
        f.Assemble()

        f.vec.data -= a.mat * uh.vec
        uh.vec.data += a.mat.Inverse(V.FreeDofs()) * f.vec
        
    return uh

In [ ]:
uh = SolveConfusion(mesh, kappa=0.3, p=3)
Draw(uh, deformation=True,
     settings={"camera" : {"transformations" :[{"type": "rotateX", "angle": -30},{"type": "rotateY", "angle": -10},{"type": "rotateZ", "angle": -10}]}});

To compare this with the exact solution, recall that (Exercise I) the exact solution 
has no $y$ dependence. So we may plot it as one-variable function of $x$ and compare it with the above multidimensional plot.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
def plotexact(kappa):
    x = np.linspace(0, 1, 1000)
    u = (1 - np.exp((x-1)/kappa)) / (1-np.exp(-1/kappa)) / 2
    plt.plot(x, u); plt.xlabel('$x$'); plt.grid(True); plt.title('Exact solution $u$')

plotexact(kappa=0.3)

Clearly, the exact solution and the discrete solution looks visually identical. Of course, to get definitive verification, we must compute the errors and its convergence rates, which is left as an exercise.

<div class="alert alert-info">
    <b><font color=red>Exercise III:</font></b>      
   Fix $\kappa = 1/3$ and computationally examine at what rate $\| u - u_h \|_{L^2(\Omega)}$ and $\| u - u_h \|_{H^1(\Omega)}$ converges to zero as $h$ becomes smaller.  (Tabulate errors on successively refined meshes and estimate the convergence rate.)  Verify that your rates are in accordance with those predicted by Exercise II.
</div>

## What happens to the discrete solution as $\kappa \to 0$?

The reason for studying this con-fusion problem is to understand when and why we do not get nice solutions from the finite element method. To this end, we compute solutions for smaller and smaller $\kappa$ while comparing the exact and discrete solutions for each $\kappa$ value. 

In [ ]:
plotexact(kappa=0.1)

In [ ]:
uh = SolveConfusion(mesh, kappa=0.1, p=3)
Draw(uh, deformation=True,
     settings={"camera" : {"transformations" :[{"type": "rotateX", "angle": -30},{"type": "rotateY", "angle": -10},{"type": "rotateZ", "angle": -10}]}});

In [ ]:
plotexact(kappa=0.01)

In [ ]:
uh = SolveConfusion(mesh, kappa=0.01, p=3)
Draw(uh, deformation=True,
     settings={"camera" : {"transformations" :[{"type": "rotateX", "angle": -30},{"type": "rotateY", "angle": -10},{"type": "rotateZ", "angle": -10}]}});

In [ ]:
plotexact(kappa=0.001)

In [ ]:
uh = SolveConfusion(mesh, kappa=0.001, p=3)
Draw(uh, deformation=True,
     settings={"camera" : {"transformations" :[{"type": "rotateX", "angle": -30},{"type": "rotateY", "angle": -10},{"type": "rotateZ", "angle": -10}]}});

As $\kappa \to 0$, the numerical solutions do not look anything like the exact solution. Oscillations seem to emanate from an increasingly thin boundary layer region and spread out over the whole domain. Such oscillations are a sign of an unstable method.

If this is confusing, do be reminded that the theory did not rule out this instability. Per an application of Cea lemma (left to Exercise II), the quasioptimality constant in the error estimate for the finite element method has a factor of $1/\kappa,$ which  goes to $\infty$ as $\kappa\to 0$, making the error estimate useless even if true.



<hr>
    
    
$\ll$ [Table Of Contents](./0_INDEX.ipynb) <br>
$\ll$ [Jay Gopalakrishnan](http://web.pdx.edu/~gjay/)

    

